In [1]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from llc import *
from generalization_measures import *
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cddinterface/Pac\
kageInfo.g:85
    if not IsKernelExtensionAvailable("CddInterface") then
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/crypting/Package\
Info.g:82
   if not IsKernelExtensionAvailable("crypting") then
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/curlinterface/Pa\
ckageInfo.g:114
  if not IsKernelExtensionAvailable("curlinterface", "curl") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/cvec/PackageInfo\
.g:114
  if not IsKernelExtensionAvailable("cvec") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/datastruct

In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [3]:
MODEL_DIR = '2024-09-23_01-06-18_A5x2_MLP2_128_wd1e-4_ubias'
disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir)
data = GroupData(params)
init_models = models[0][:5]
models = models[-1][:5]

  0%|          | 0/5 [00:00<?, ?it/s]/home/wilson/Finite-groups/src/model_utils.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path,

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: smallgrp(120,35)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [27]:
measures = get_gen_measures(init_models, models, data)

MARGIN
NORMS
PATH_NORM
PACBAYES


  0%|          | 0/15 [00:00<?, ?it/s]

PACBAYES_MAG


  0%|          | 0/15 [00:00<?, ?it/s]

WARN: Binary search with depth 15 failed to converge for 2 instances!


In [29]:
measures

{'L2': tensor([0.0963, 0.0957, 0.0962, 0.0957, 0.1021]),
 'L2_DIST': tensor([0.1764, 0.1769, 0.1770, 0.1766, 0.1767]),
 'INVERSE_MARGIN': tensor([0.0296, 0.0318, 0.0291, 0.0325, 0.0601]),
 'LOG_PROD_OF_SPEC': tensor([6.4720, 7.3142, 6.4325, 7.3142, 6.2487]),
 'LOG_PROD_OF_SPEC_OVER_MARGIN': tensor([4.7112, 5.5900, 4.6644, 5.6002, 4.8424]),
 'LOG_SPEC_INIT_MAIN': tensor([8.5069, 9.3856, 8.4600, 9.3959, 8.6381]),
 'FRO_OVER_SPEC': tensor([0.0816, 0.0668, 0.0821, 0.0668, 0.0944]),
 'LOG_SPEC_ORIG_MAIN': tensor([6.5343, 7.2131, 6.4936, 7.2228, 6.8115]),
 'LOG_SUM_OF_SPEC_OVER_MARGIN': tensor([-1.7165, -1.5408, -1.7259, -1.5387, -1.6903]),
 'LOG_SUM_OF_SPEC': tensor([-1.3644, -1.1959, -1.3723, -1.1959, -1.4090]),
 'LOG_PROD_OF_FRO': tensor([11.4237, 11.3645, 11.4157, 11.3621, 12.0177]),
 'LOG_PROD_OF_FRO_OVER_MARGIN': tensor([ 9.6629,  9.6402,  9.6476,  9.6481, 10.6114]),
 'LOG_SUM_OF_FRO_OVER_MARGIN': tensor([-0.7262, -0.7307, -0.7292, -0.7291, -0.5365]),
 'LOG_SUM_OF_FRO': tensor([-0.3740

In [24]:
LLC_PARAMS = {
    'eps': 8e-4,
    'beta': 1e3,
    'gamma': 1.,
    'chains': 5,
    'cbatch_size': 5,
    'replacement': False,
    'behavioral': False,
    # 'epochs': 3000,
    'epochs': 30,
}
measures['LLC'] = get_llc(models, data, **LLC_PARAMS)[0].detach()

SGLD:   0%|          | 0/30 [00:00<?, ?it/s]

In [25]:
measures['LLC']

tensor([592.0754, 491.7903, 605.9983, 536.8795, 741.5655])

In [4]:
from torch.func import jvp, grad, vjp, vmap, functional_call

# Adapted from https://github.com/akshayka/hessian_trace_estimation/blob/main/hessian_trace_estimation.ipynb

class LinearOperator:
    def __init__(self, matvec):
        self._matvec = matvec

    def matvec(self, vecs):
        return self._matvec(vecs)

def hutchpp(A, d, m):
    """https://arxiv.org/abs/2010.09649

    A is the LinearOperator whose trace to estimate
    d is the input dimension
    m is the number of queries (larger m yields better estimates)
    """
    S = t.randn(d, m // 3)
    G = t.randn(d, m // 3)
    AS = A.matvec(S)
    Q, _ = t.qr(AS)
    proj = G - Q @ (Q.T @ G)
    return t.trace(Q.T @ A.matvec(Q)) + (3./m)*t.trace(proj.T @ A.matvec(proj))

def make_hvp(f, x):
    def hvp(f, primals, tangents):
        return vmap(lambda t: jvp(grad(f), (primals, ), (t, ))[1], in_dims=1, out_dims=1)(tangents)

    return LinearOperator(lambda v: hvp(f, x, v))

In [12]:
from typing import Dict, Tuple

def flatten(
    params: Dict[str, t.Tensor]
) -> Tuple[t.Tensor, Dict[str, t.Size]]:
    return t.cat([p.flatten() for p in params.values()]), {k: p.size() for k, p in params.items()}

def unflatten(
    flat_params: t.Tensor, 
    shapes: Dict[str, t.Size],
) -> Dict[str, t.Tensor]:
    params = dict()
    for name, shape in shapes.items():
        size = np.prod(shape)
        params[name] = flat_params[:size].reshape(shape)
        flat_params = flat_params[size:]
    return params

def h_tr(models, data, m=100):
    ret = []
    train_dataset = t.tensor(data.train_data, device=device)

    for i in tqdm(range(len(models))):
        model = models[i]
        flat_init_params, shapes = flatten(dict(model.named_parameters()))
        def loss_func(flat_params, model=model, shapes=shapes, dataset=train_dataset):
            x, z = dataset[:, :-1], dataset[:, -1]
            output = t.func.functional_call(model, unflatten(flat_params, shapes), x)
            return get_cross_entropy(output, z).squeeze()
        
        model_hvp = make_hvp(loss_func, flat_init_params)
        ret.append(hutchpp(model_hvp, flat_init_params.size(0), m))
    return t.tensor(ret)

In [14]:
h_tr(models, data)

  0%|          | 0/5 [00:00<?, ?it/s]

tensor([ 9.7015, 13.2429,  9.6771, 13.6952, 20.3457])

In [8]:
H

tensor(9.8350, grad_fn=<AddBackward0>)